In [77]:
import requests,re,os,json
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from openai import OpenAI
from typing import List
from pydantic import BaseModel
def html_to_soup(url):
	try:
		res = requests.get('https://m.prts.wiki'+url, timeout=(5, 5))
	except requests.exceptions.Timeout:
		return None
	soup = BeautifulSoup(res.text, 'html.parser')
	return soup

def html_to_soup_saved(url):
	soup_filename = make_filename(url,'txt')
	soup_path = './soup/soup'+soup_filename
	flag = 1
	if os.path.exists(soup_path):
		with open(soup_path,'r') as f:
			h = f.read()
		if h: # 以前記録したのが空でなければ，それを使う
			soup = BeautifulSoup(h,'html.parser')
			flag = 0
	if flag:
		print(title+' fetching..')
		soup = html_to_soup(url)
		if soup:
			with open(soup_path,'w') as f:
				f.write(soup.find('script', {'id': 'datas_txt', 'type': 'csv'}).prettify())
	return soup

# soup -> df
def make_df(soup):
	script = soup.find('script', {'id': 'datas_txt', 'type': 'csv'})
	lines = [line.strip() for line in script.string.splitlines() if line.strip()]
	records = []
	pattern = re.compile(
		r'^\[(?P<cmd>\w+)'
		r'(?:\((?P<args>.*?)\)|=(?P<val>[^]]+))?'
		r'\](?:\s*(?P<text>.*))?$'
	)
	for ln in lines:
		m = pattern.match(ln)
		if m:
			cmd   = m.group('cmd').lower()
			args  = m.group('args')
			val   = m.group('val')
			text  = m.group('text') or ''

			params = {}
			if args:
				for part in re.split(r',\s*(?=\w+=)', args):
					k, v = part.split('=', 1)
					params[k] = v.strip().strip('"')
			elif val is not None:
				params[cmd] = val.strip('"')

			records.append({'command': cmd, 'text': text, **params})
		else:
			records.append({'command': None, 'text': ln})
	# DataFrame に
	df = pd.DataFrame(records)
	df['text'] = df['text'].str.replace('Dr.{@nickname}', '博士')
	df['text'] = df['text'].str.replace('Dr. {@nickname}', '博士')
	df['text'] = df['text'].str.replace('{@nickname}博士', '博士')
	if 'name' not in df.columns:
		df['name'] = ''


	if 'options' in df.columns:
		df['option_number'] = (
			df['options']
			.fillna('')
			.apply(lambda x: len(x.split(';')) if x.strip() else 0)
		)
		df = df.reset_index().rename(columns={'index':'orig_idx'})
		mask = ((df['text'].eq('') | df['text'].isna()) &
		        df['options'].astype(str).str.strip().ne(''))
		df_expand = (
			df[mask]
			# 1) options／values をリスト化
			.assign(
				opts = lambda d: d['options'].fillna('').str.split(';'),
				vals = lambda d: d['values'].fillna('').str.split(';')
			)
			# 2) zip で (opt, val) のタプルリストを作成→自動で余分な要素を切り捨て
			.assign(pairs = lambda d: [
				list(zip(o, v)) for o, v in zip(d['opts'], d['vals'])
			])
			# 3) explode＆展開
			.explode('pairs')
			.assign(
				text         = lambda d: d['pairs'].str[0],
				option_value = lambda d: d['pairs'].str[1],
				option_order = lambda d: d.groupby(level=0).cumcount()
			)
			# 4) いらない列を削除
			.drop(columns=['opts','vals','pairs'])
		)


		df_no = df[~mask].copy()
		df_no['option_order'] = pd.NA  # ソート用に同じ列を用意

		df = (
			pd.concat([df_expand, df_no], ignore_index=True)
			.sort_values(['orig_idx','option_order'], kind='stable')
			.reset_index(drop=True)
			# 不要列を落とす
			.drop(['orig_idx','option_order','options'], axis=1)
		)

	keep_cols = ['command','text','name','option_number','option_value','references','focus','image','index','values','name1','name2','name3','name4']
	df = df[[col for col in keep_cols if col in df.columns]]
	return df

# soup,df -> キャラのurlのdict
def char_img_dict(soup):
	datas_char= soup.find('script', {'id': 'datas_char', 'type': 'csv'}).string
	datas_char =  pd.read_csv(StringIO(datas_char), header=None)
	datas_char.columns = ['name', 'html']
	datas_char['name'] = datas_char['name'].str.lower()
	datas_char = dict(zip(datas_char['name'], datas_char['html']))
	return datas_char

client = OpenAI()
class TranslateAllModel(BaseModel):
	class TranslateModel(BaseModel):
		index: int
		name: str
		jp_name:str
		text: str
		jp_text: str
	translation: List[TranslateModel]

def translate(df,mini):
	df["index"] = df.index
	p = df[pd.notna(df['text'])]
	p = p[['index','name','text']]

	if mini:model = 'gpt-4.1-mini'
	else:model = 'gpt-4.1'

	chunk_size = 50
	chunks = [p.iloc[i : i + chunk_size] for i in range(0, len(p), chunk_size)]
	processed = []
	for chunk in chunks:
		p = chunk.to_json(orient='records',force_ascii=False)
		prompt = json.dumps(p, ensure_ascii=False)
		response = client.responses.parse(
			model=model,
			input=[
				{"role": "system", "content":
					"""あなたは優秀な翻訳者です。以下のリスト形式の JSON を読み込み、各オブジェクトの 'name',“text” を日本語に翻訳し、キー “jp_name”, “jp_text” として追加してください。出力は同じリスト形式の JSON のまま返してください。
					# 注意　「博士」は「ドクター」と訳すこと。入力が「？」など記号のみの場合，そのまま出力せよ。阿米娅のセリフは敬体にしなさい。
					＃　固有名詞　'塔露拉': タルラ, '凯尔希': ケルシー, '阿丽娜': アリーナ,'爱国者':パトリオット"""},
				{"role": "user",   "content": prompt},
			],
			text_format=TranslateAllModel,
		)
		processed_chunk = pd.DataFrame(response.output_parsed.dict()['translation'])
		processed.append(processed_chunk)

	# 処理結果を結合
	trans_df =  pd.concat(processed, ignore_index=True)
	trans_df['text'] = trans_df['text'].str.replace('\\\\', '\\')
	trans_df['text'] = trans_df['text'].str.replace('\\"', '"')

	merged = (df.merge(
		trans_df[["index", "text", "jp_text",'jp_name']],  # 原文(text), 翻訳(jp_text)を取り込む
		on="index",
		how="left",
		suffixes=("_orig", ""))
	          .assign(text_match=lambda d: d["text"] == d["text_orig"]  # text と text_orig が同じか　->　text_match列に
	                  )
	          )
	m = merged[~merged["text_match"]][["index", "text",'text_orig']]
	k = m[m["text"].notna()& m["text"].str.strip().ne("")]
	if k.size:
		# print('不一致')
		print(k)
	# return None
	return merged

def make_filename(url, extension='html'):
	filename_list = url.split('/')[2:]
	filename = filename_list[0].split('_')[0]
	if len(filename_list)>1:
		filename += '-'+filename_list[1]
	if extension=='html':
		filename += '.html'
	if extension=='csv':
		filename += '.csv'
	if extension=='txt':
		filename += '.txt'
	return filename

def url_to_translatedCSV(url,title,mini,force_translate=False):
	soup_filename = make_filename(url,'txt')
	soup_path = './soup/soup'+soup_filename
	flag = 1
	soup = ''
	if os.path.exists(soup_path):
		with open(soup_path,'r') as f:
			h = f.read()
		if h: # 以前記録したのが空でなければ，それを使う
			soup = BeautifulSoup(h,'html.parser')
			flag = 0
	if flag:
		soup = html_to_soup(url)
		if soup:
			with open(soup_path,'w') as f:
				f.write(soup.find('script', {'id': 'datas_txt', 'type': 'csv'}).prettify())
	csv_filename = make_filename(url,'csv')
	exist_csv = ''
	if mini:
		csv_path = './csv/mini/'+csv_filename
		if os.path.exists('./csv/'+csv_filename):
			exist_csv = './csv/'+csv_filename
		elif os.path.exists(csv_path):
			exist_csv = csv_path
	else:
		csv_path = './csv/'+csv_filename
		if os.path.exists(csv_path):
			exist_csv = csv_path
	if not soup:
		print('timeout')
		return
	elif not exist_csv or force_translate:
		df = make_df(soup)
		df = translate(df,mini)
		df.to_csv(csv_path, index=False)

In [78]:
def text_html(pp,icon='',translation=True):
	bbb = ''
	if pp['command'] == 'name':
		if pd.notna(pp['name']) and pp['name']!='':
			bbb += f'{icon}\n<em class="name">{pp["name"]}</em>\n'
			if translation and pp['jp_name']!='':
				bbb += f'<em class="translation">{pp["jp_name"]}</em>\n'

		bbb = f'<div class="character">{bbb}</div>'
		text = ''
		if pd.notna(pp['text'])and pp['text']!='':
			text += f'<p>{pp["text"]}</p>\n'
			if translation and pp['jp_text']!='':
				text += f'<p class="translation">{pp["jp_text"]}</p>\n'
			bbb += f"""<div class="text">\n{text}</div>"""
		return f"""<li class="dialogue">\n{bbb}\n</li>\n"""
	else:
		return bbb

def icon_html(pp,char_dict):
	a =  'name'
	if 'focus' in pp.index and pd.notna(pp['focus']):
		f = int(pp['focus'])
		if f>1:
			a += str(f)
	d = {'char_136_hsguma':'char_136_hsguma_1',
	     'char_1504_cqbw':'char_1504_cqbw_1',
	     'char_2006_weiywfmzuki_1':"char_2006_fmzuki_1"}
	cha_id = pp[a].split('#')[0].lower()

	if cha_id in d.keys():
		cha_id = d[cha_id]
	if cha_id in char_dict.keys():
		url = char_dict[cha_id]
		a = f'<div class="icon" style="--img-url: url(\'{url}\')" data-src="{url}"></div>'
		return a
	else: return None

def img_back_html(image_id, df_datas_back):
	t = df_datas_back[df_datas_back['name']==image_id].html.iloc[0]
	return f'<img src="{t}" width="100%" height="auto">'

from collections import defaultdict
# soup,df -> html（本文）
def make_html(df,translation=True):
	p = df
	b = ''
	with open('soup/soup14-8-BEG.txt','r') as f:
		h = f.read()
	soup = BeautifulSoup(h,'html.parser')
	datas_back = soup.find('script', {'id': 'datas_back', 'type': 'csv'}).string
	df_datas_back=pd.read_csv(StringIO(datas_back), header=None)
	df_datas_back.columns = ['name', 'html']
	char_dict = char_img_dict(soup)

	# watched_img = []

	opts_dict = defaultdict(str)
	opts_dict_jp = defaultdict(str)
	icon = ''

	for i in range(p.shape[0]):
		pp = p.iloc[i]
		c = pp['command']
		if c == 'decision':
			op_id = int(pp['option_value'])
			if op_id==1: # 連続するdecisionのうち最初のもの
				b += '<div class="decision"><li class="decision">'
				b += f'<p class="decision">「{pp["text"]}」</p>'
				if translation: b += f'<p class="translation">「{pp["jp_text"]}」</p>'

			elif op_id == int(pp['option_number']):
				b += f'<p class="decision">「{pp["text"]}」</p>'
				if translation: b += f'<p class="translation">「{pp["jp_text"]}」</p>'
				b += '</li>'
			else:
				b += f'<p class="decision">「{pp["text"]}」</p>'
				if translation: b+= f'<p class="translation">「{pp["jp_text"]}」</p>'
			opts_dict[op_id] = pp['text']
			opts_dict_jp[op_id] = pp['jp_text']
			opts_dict['option_number']=int(pp['option_number'])

		elif c=='predicate':
			if opts_dict['option_number']==1:
				b += '</div>'
				opts_dict = defaultdict(str)
				opts_dict_jp = defaultdict(str)
			else:
				ref = pp['references']
				if type(ref)==str and ';' not in ref:
					ref = int(ref)
					if ref>1: b += '</div>'
					b += '<div class="predicate">'
					b +=  f'<p>「{opts_dict[ref]}」</p>'
					if translation: b += f'<p class="translation">「{opts_dict_jp[ref]}」</p>'

				else:  # ; が含まれる時

					b += '</div></div>'
					opts_dict = defaultdict(str)
					opts_dict_jp = defaultdict(str)
		elif c=='character':
			if pd.notna(pp['name']) and pp['name']!='':
				icon = icon_html(pp,char_dict)
			else:
				icon = ''
			if  ('focus' in pp.index) and pp['focus']==-1:
				icon = ""
		elif c=='blocker':
			icon=''
		elif c=='name':
			b += text_html(pp,icon,translation)
		elif c in ('image','background'):
			image_id = pp['image']
			if pd.notna(image_id):
				if c=='background':
					image_id = 'bg_' + image_id
				b += img_back_html(image_id, df_datas_back)+'\n'
		elif pd.notna(pp['text']) and pp['text']!='' and c!='header':
			b += f'<p>{pp["text"]}</p>\n'
			if translation and pp['jp_text']!='':
				b+= f'<p class="translation">{pp["jp_text"]}</p>\n'
	return b

In [82]:
def check_translation_slip(csv_path,df=None):
	if df is None:
		df = pd.read_csv(csv_path)
	non_jp = df[df['command'].isna() & df['text'].notna() & df['jp_text'].isna()]
	if not non_jp.empty:
		print(csv_path,f'translate error detected.{non_jp.shape[0]}lines. now translating..')

	for idx, row in non_jp.iterrows():
		orig = row['text']
		response = client.responses.create(
			model="gpt-4.1-mini",
			input=[{
				'role': 'system',
				'content': 'あなたは翻訳家です。中国語を与えるので自然な日本語に訳してください。英語のみのときや記号のみの場合はそのまま出力せよ。そのほかの出力は絶対にするな',
			},
				{'role':'user','content':orig}]
		)
		df.loc[idx, 'jp_text'] = response.output_text
	keep_cols = ['command','text','name','option_number','option_value','references','focus','image','index','values','name2','name3','name4','name5','name6','jp_text','jp_name']
	df = df[[col for col in keep_cols if col in df.columns]]
	df.to_csv(csv_path, index=False)
	return df

In [83]:
def translate_title(title):
	appendix = ''
	if '行动前' == title[-3:]:
		title = title[:-3]
		appendix = ' 戦闘前'
	elif '行动后' == title[-3:]:
		title = title[:-3]
		appendix = ' 戦闘後'
	response = client.responses.create(
		model="gpt-4.1-mini",
		input=[{
			'role': 'system',
			'content': 'あなたは中国語の翻訳家です。中国語の小説のタイトルを与えるので，日本語に訳してください。英語のみのときや記号のみの場合はそのまま出力せよ。そのほかの出力は絶対にするな',
		},
			{'role':'user','content':title}]
	)
	return response.output_text + appendix

def make_html_all(url,title,next_page='',translation=True,mini=False,update_html=False, force_translate=False):
	if not update_html:
		if os.path.exists('./html/'+make_filename(url)):
			# print('html already exists')
			return
	soup = html_to_soup_saved(url)

	csv_filename = make_filename(url,'csv')
	exist_csv = ''
	if mini:
		csv_path = './csv/mini/'+csv_filename
		if os.path.exists('./csv/'+csv_filename):
			exist_csv = './csv/'+csv_filename
		elif os.path.exists(csv_path):
			exist_csv = csv_path
	else:
		csv_path = './csv/'+csv_filename
		if os.path.exists(csv_path):
			exist_csv = csv_path

	df = make_df(soup)
	# csv がある時
	if exist_csv and not force_translate:
		df_t = pd.read_csv(exist_csv)
		df_t_valid = df_t[df_t['text'].notna()]
		# ２．同じ text が複数ある場合は最初の行だけ残す
		df_t_unique = df_t_valid.drop_duplicates(subset='text')
		# 方法１：merge で一度に両方追加
		df = df.merge(
			df_t_unique[['text', 'jp_text', 'jp_name']],
			on='text',
			how='left')
		dup_mask = df['text'].duplicated()
		df.loc[dup_mask, 'jp_name'] = ''
		df=check_translation_slip(exist_csv,df)
	# csvないとき　翻訳のcsvを新規作成する
	else:
		if not soup:
			print('timeout')
			return
		print(title+' fetched. now translating..')
		if translation:
			df = translate(df,mini)
		df.to_csv(csv_path, index=False)

	body = make_html(df,translation=translation)
	title = f'{title}　-　{translate_title(title)}'
	html = (f"""<!DOCTYPE html>
	<html lang="zh-CN">
	<head>
	 <meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
	 <script src="https://kit.fontawesome.com/c07fe94aba.js" crossorigin="anonymous"></script>
	 <link href="https://fonts.googleapis.com/css2?family=Noto+Sans:wght@400;700&display=swap" rel="stylesheet">
	 <title>{title}</title>
	 <link rel="stylesheet" href="../css/styles.css">
</head><body><h2 class="title">{title}</h2>
<ul>{body}</ul>  <div class="button-container">
   <button class="back-home" onclick="location.href='../index.html'" aria-label="ホーム">
    <i class="fa-solid fa-house"></i>
   </button>
  <button class="next-page" onclick="location.href='{next_page}'">
   <i class="fa-solid fa-angle-right"></i>
  </button>
  </div>
 <div class="overlay">
   <img class="overlay-img" src="" alt="元画像">
  </div>
<script>
	const overlay = document.querySelector('.overlay');
    const img = overlay.querySelector('.overlay-img');

    document.body.addEventListener('click', e => {{
      if (e.target.matches('.icon')) {{
        img.src = e.target.dataset.src;
        overlay.classList.add('show');
      }}
      else if (e.target === overlay) {{
        overlay.classList.remove('show');
      }}    }});
        function fitTextToWidth(el, maxWidth) {{
    // 現在のフォントサイズを取得
    let style = window.getComputedStyle(el);
	let fontSize = parseFloat(style.fontSize);
	// 幅がオーバーしている限り、1pxずつ小さくする
while (el.scrollWidth > maxWidth && fontSize > 8) {{
fontSize -= 1;
el.style.fontSize = fontSize + 'px';
}}
}}

// ページ読み込み後／ウィンドウリサイズ時に実行
function adjustAll() {{
document.querySelectorAll('.character').forEach(char => {{
	const maxW = char.clientWidth;
char.querySelectorAll('em.name, em.translation').forEach(el => {{
// 初期サイズにリセットしてから再計算
el.style.fontSize = '';
fitTextToWidth(el, maxW);
}});
}});
}}

window.addEventListener('load', adjustAll);
window.addEventListener('resize', adjustAll);
</script>
</body>
</html>""")
	soup = BeautifulSoup(html, 'html.parser')
	for el in soup.select('.translation'):
		el['lang'] = 'jp'
	html = soup.prettify()
	filename = make_filename(url)
	with open('./html/'+filename,'w',encoding='utf-8') as f:
		f.write(html)
	print(f"⇒ {title} saved to {filename}")

In [85]:
links = [
	['7-132:00:00', '/w/7-1_32:00:00/NBT'], ['7-2别离之夜行动前', '/w/7-2_%E5%88%AB%E7%A6%BB%E4%B9%8B%E5%A4%9C/BEG'], ['7-2别离之夜行动后', '/w/7-2_%E5%88%AB%E7%A6%BB%E4%B9%8B%E5%A4%9C/END'], ['7-3变节之刃行动前', '/w/7-3_%E5%8F%98%E8%8A%82%E4%B9%8B%E5%88%83/BEG'], ['7-3变节之刃行动后', '/w/7-3_%E5%8F%98%E8%8A%82%E4%B9%8B%E5%88%83/END'], ['7-4并肩之约-1行动前', '/w/7-4_%E5%B9%B6%E8%82%A9%E4%B9%8B%E7%BA%A6-1/BEG'], ['7-5并肩之约-2行动后', '/w/7-5_%E5%B9%B6%E8%82%A9%E4%B9%8B%E7%BA%A6-2/END'], ['7-6遗忘之地行动前', '/w/7-6_%E9%81%97%E5%BF%98%E4%B9%8B%E5%9C%B0/BEG'], ['7-6遗忘之地行动后', '/w/7-6_%E9%81%97%E5%BF%98%E4%B9%8B%E5%9C%B0/END'], ['7-726:37:14', '/w/7-7_26:37:14/NBT'], ['7-8沉默者之怒-1行动前', '/w/7-8_%E6%B2%89%E9%BB%98%E8%80%85%E4%B9%8B%E6%80%92-1/BEG'], ['7-9沉默者之怒-2行动后', '/w/7-9_%E6%B2%89%E9%BB%98%E8%80%85%E4%B9%8B%E6%80%92-2/END'], ['7-10暗淡者之火行动前', '/w/7-10_%E6%9A%97%E6%B7%A1%E8%80%85%E4%B9%8B%E7%81%AB/BEG'], ['7-10暗淡者之火行动后', '/w/7-10_%E6%9A%97%E6%B7%A1%E8%80%85%E4%B9%8B%E7%81%AB/END'], ['7-13感染者之盾-1行动前', '/w/7-13_%E6%84%9F%E6%9F%93%E8%80%85%E4%B9%8B%E7%9B%BE-1/BEG'], ['7-17感染者之盾-2行动后', '/w/7-17_%E6%84%9F%E6%9F%93%E8%80%85%E4%B9%8B%E7%9B%BE-2/END'], ['7-18爱国者之死行动前', '/w/7-18_%E7%88%B1%E5%9B%BD%E8%80%85%E4%B9%8B%E6%AD%BB/BEG'], ['7-18爱国者之死行动后', '/w/7-18_%E7%88%B1%E5%9B%BD%E8%80%85%E4%B9%8B%E6%AD%BB/END'], ['7-1911:15:38', '/w/7-19_11:15:38/NBT'], ['7-20??:??:??', '/w/7-20_%3F%3F:%3F%3F:%3F%3F/NBT'], ['R8-1昨日，谷壳将裂行动前', '/w/R8-1_%E6%98%A8%E6%97%A5%EF%BC%8C%E8%B0%B7%E5%A3%B3%E5%B0%86%E8%A3%82/BEG'], ['R8-1昨日，谷壳将裂行动后', '/w/R8-1_%E6%98%A8%E6%97%A5%EF%BC%8C%E8%B0%B7%E5%A3%B3%E5%B0%86%E8%A3%82/END'], ['M8-1今日，血色满溢', '/w/M8-1_%E4%BB%8A%E6%97%A5%EF%BC%8C%E8%A1%80%E8%89%B2%E6%BB%A1%E6%BA%A2/NBT'], ['R8-3麦秆，极易燃烧行动前', '/w/R8-3_%E9%BA%A6%E7%A7%86%EF%BC%8C%E6%9E%81%E6%98%93%E7%87%83%E7%83%A7/BEG'], ['R8-3麦秆，极易燃烧行动后', '/w/R8-3_%E9%BA%A6%E7%A7%86%EF%BC%8C%E6%9E%81%E6%98%93%E7%87%83%E7%83%A7/END'], ['M8-2失语，产自多言', '/w/M8-2_%E5%A4%B1%E8%AF%AD%EF%BC%8C%E4%BA%A7%E8%87%AA%E5%A4%9A%E8%A8%80/NBT'], ['R8-4火种，一触即灭行动前', '/w/R8-4_%E7%81%AB%E7%A7%8D%EF%BC%8C%E4%B8%80%E8%A7%A6%E5%8D%B3%E7%81%AD/BEG'], ['R8-4火种，一触即灭行动后', '/w/R8-4_%E7%81%AB%E7%A7%8D%EF%BC%8C%E4%B8%80%E8%A7%A6%E5%8D%B3%E7%81%AD/END'], ['M8-3死亡，召之即来', '/w/M8-3_%E6%AD%BB%E4%BA%A1%EF%BC%8C%E5%8F%AC%E4%B9%8B%E5%8D%B3%E6%9D%A5/NBT'], ['R8-5寒冷，来自知觉行动前', '/w/R8-5_%E5%AF%92%E5%86%B7%EF%BC%8C%E6%9D%A5%E8%87%AA%E7%9F%A5%E8%A7%89/BEG'], ['R8-5寒冷，来自知觉行动后', '/w/R8-5_%E5%AF%92%E5%86%B7%EF%BC%8C%E6%9D%A5%E8%87%AA%E7%9F%A5%E8%A7%89/END'], ['M8-4意志，片缕幻影', '/w/M8-4_%E6%84%8F%E5%BF%97%EF%BC%8C%E7%89%87%E7%BC%95%E5%B9%BB%E5%BD%B1/NBT'], ['R8-6战场，蔓延不止行动前', '/w/R8-6_%E6%88%98%E5%9C%BA%EF%BC%8C%E8%94%93%E5%BB%B6%E4%B8%8D%E6%AD%A2/BEG'], ['R8-6战场，蔓延不止行动后', '/w/R8-6_%E6%88%98%E5%9C%BA%EF%BC%8C%E8%94%93%E5%BB%B6%E4%B8%8D%E6%AD%A2/END'], ['M8-5厄运，等候已久', '/w/M8-5_%E5%8E%84%E8%BF%90%EF%BC%8C%E7%AD%89%E5%80%99%E5%B7%B2%E4%B9%85/NBT'], ['R8-8人心，向背无常行动前', '/w/R8-8_%E4%BA%BA%E5%BF%83%EF%BC%8C%E5%90%91%E8%83%8C%E6%97%A0%E5%B8%B8/BEG'], ['R8-8人心，向背无常行动后', '/w/R8-8_%E4%BA%BA%E5%BF%83%EF%BC%8C%E5%90%91%E8%83%8C%E6%97%A0%E5%B8%B8/END'], ['M8-6再见，只为再见行动前', '/w/M8-6_%E5%86%8D%E8%A7%81%EF%BC%8C%E5%8F%AA%E4%B8%BA%E5%86%8D%E8%A7%81/BEG'], ['M8-6再见，只为再见行动后', '/w/M8-6_%E5%86%8D%E8%A7%81%EF%BC%8C%E5%8F%AA%E4%B8%BA%E5%86%8D%E8%A7%81/END'], ['R8-9相逢，总是离别行动前', '/w/R8-9_%E7%9B%B8%E9%80%A2%EF%BC%8C%E6%80%BB%E6%98%AF%E7%A6%BB%E5%88%AB/BEG'], ['R8-9相逢，总是离别行动后', '/w/R8-9_%E7%9B%B8%E9%80%A2%EF%BC%8C%E6%80%BB%E6%98%AF%E7%A6%BB%E5%88%AB/END'], ['M8-7恶言，报应不爽行动前', '/w/M8-7_%E6%81%B6%E8%A8%80%EF%BC%8C%E6%8A%A5%E5%BA%94%E4%B8%8D%E7%88%BD/BEG'], ['M8-7恶言，报应不爽行动后', '/w/M8-7_%E6%81%B6%E8%A8%80%EF%BC%8C%E6%8A%A5%E5%BA%94%E4%B8%8D%E7%88%BD/END'], ['R8-11落雪，浸黑国土行动前', '/w/R8-11_%E8%90%BD%E9%9B%AA%EF%BC%8C%E6%B5%B8%E9%BB%91%E5%9B%BD%E5%9C%9F/BEG'], ['R8-11落雪，浸黑国土行动后', '/w/R8-11_%E8%90%BD%E9%9B%AA%EF%BC%8C%E6%B5%B8%E9%BB%91%E5%9B%BD%E5%9C%9F/END'], ['M8-8苏醒，浮出梦乡行动前', '/w/M8-8_%E8%8B%8F%E9%86%92%EF%BC%8C%E6%B5%AE%E5%87%BA%E6%A2%A6%E4%B9%A1/BEG'], ['M8-8苏醒，浮出梦乡行动后', '/w/M8-8_%E8%8B%8F%E9%86%92%EF%BC%8C%E6%B5%AE%E5%87%BA%E6%A2%A6%E4%B9%A1/END'], ['JT8-1恨火，流向原野行动前', '/w/JT8-1_%E6%81%A8%E7%81%AB%EF%BC%8C%E6%B5%81%E5%90%91%E5%8E%9F%E9%87%8E/BEG'], ['JT8-1恨火，流向原野行动后', '/w/JT8-1_%E6%81%A8%E7%81%AB%EF%BC%8C%E6%B5%81%E5%90%91%E5%8E%9F%E9%87%8E/END'], ['JT8-2睁眼，便是日暮行动前', '/w/JT8-2_%E7%9D%81%E7%9C%BC%EF%BC%8C%E4%BE%BF%E6%98%AF%E6%97%A5%E6%9A%AE/BEG'], ['JT8-2睁眼，便是日暮行动后', '/w/JT8-2_%E7%9D%81%E7%9C%BC%EF%BC%8C%E4%BE%BF%E6%98%AF%E6%97%A5%E6%9A%AE/END'], ['JT8-3昂首，足践烈焰行动前', '/w/JT8-3_%E6%98%82%E9%A6%96%EF%BC%8C%E8%B6%B3%E8%B7%B5%E7%83%88%E7%84%B0/BEG'], ['JT8-3昂首，足践烈焰行动后', '/w/JT8-3_%E6%98%82%E9%A6%96%EF%BC%8C%E8%B6%B3%E8%B7%B5%E7%83%88%E7%84%B0/END'], ['END8-1尾声，抑或开始', '/w/END8-1_%E5%B0%BE%E5%A3%B0%EF%BC%8C%E6%8A%91%E6%88%96%E5%BC%80%E5%A7%8B/NBT'], ['EG-1燃烧的片段1', '/w/EG-1_%E7%87%83%E7%83%A7%E7%9A%84%E7%89%87%E6%AE%B51/NBT'], ['EG-2燃烧的片段2', '/w/EG-2_%E7%87%83%E7%83%A7%E7%9A%84%E7%89%87%E6%AE%B52/NBT'], ['EG-3燃烧的片段3', '/w/EG-3_%E7%87%83%E7%83%A7%E7%9A%84%E7%89%87%E6%AE%B53/NBT'], ['EG-4燃烧的片段4', '/w/EG-4_%E7%87%83%E7%83%A7%E7%9A%84%E7%89%87%E6%AE%B54/NBT'], ['EG-5燃烧的片段5', '/w/EG-5_%E7%87%83%E7%83%A7%E7%9A%84%E7%89%87%E6%AE%B55/NBT'],]
for i in range(len(links)):
	title, url = links[i]
	if i<len(links)-1:
		_,next_p = links[i+1]
		next_p = make_filename(next_p)
	try:
		make_html_all(url,title,next_page=next_p,translation=True,mini=True,update_html=False,force_translate=False)
	except Exception as e:
		print(title,e)

⇒ R8-8人心，向背无常行动前　-　人の心は移ろいやすい 戦闘前 saved to R8-8-BEG.html
⇒ R8-8人心，向背无常行动后　-　人心の向きは常ならず 戦闘後 saved to R8-8-END.html
⇒ M8-6再见，只为再见行动前　-　M8-6さよなら、たださよならのために 戦闘前 saved to M8-6-BEG.html
⇒ M8-6再见，只为再见行动后　-　M8-6さよなら、ただ再会のために 戦闘後 saved to M8-6-END.html
⇒ R8-9相逢，总是离别行动前　-　R8-9出会いはいつも別れ 戦闘前 saved to R8-9-BEG.html
⇒ R8-9相逢，总是离别行动后　-　R8-9出会いは、いつも別れ 戦闘後 saved to R8-9-END.html
⇒ M8-7恶言，报应不爽行动前　-　M8-7悪言、報いは逃れられない 戦闘前 saved to M8-7-BEG.html
⇒ M8-7恶言，报应不爽行动后　-　M8-7悪言、報いは逃れられない 戦闘後 saved to M8-7-END.html
⇒ R8-11落雪，浸黑国土行动前　-　R8-11落雪，浸黑国土 戦闘前 saved to R8-11-BEG.html
⇒ R8-11落雪，浸黑国土行动后　-　R8-11落雪，浸黑国土 戦闘後 saved to R8-11-END.html
⇒ M8-8苏醒，浮出梦乡行动前　-　M8-8 目覚め、夢の中から浮かび上がる 戦闘前 saved to M8-8-BEG.html
M8-8苏醒，浮出梦乡行动后 fetched. now translating..
⇒ M8-8苏醒，浮出梦乡行动后　-　M8-8 目覚め、夢の国から浮上 戦闘後 saved to M8-8-END.html
./csv/mini/JT8-1-BEG.csv translate error detected.6lines. now translating..
⇒ JT8-1恨火，流向原野行动前　-　恨みの炎、野原へ流れる 戦闘前 saved to JT8-1-BEG.html
./csv/mini/JT8-1-END.csv translate error detected.5lines. now translating..
⇒ J

In [ ]:
c = [[['6-12冰原之雪行动前', '/w/6-12_%E5%86%B0%E5%8E%9F%E4%B9%8B%E9%9B%AA/BEG'], ['6-13没有火,没有光', '/w/6-13_%E6%B2%A1%E6%9C%89%E7%81%AB,%E6%B2%A1%E6%9C%89%E5%85%89/NBT']],

      [['6-13没有火,没有光', '/w/6-13_%E6%B2%A1%E6%9C%89%E7%81%AB,%E6%B2%A1%E6%9C%89%E5%85%89/NBT'], ['6-14冰原之霜行动后', '/w/6-14_%E5%86%B0%E5%8E%9F%E4%B9%8B%E9%9C%9C/END']],


      [['6-14冰原之霜行动后', '/w/6-14_%E5%86%B0%E5%8E%9F%E4%B9%8B%E9%9C%9C/END'], ['6-15不错的回忆行动前', '/w/6-15_%E4%B8%8D%E9%94%99%E7%9A%84%E5%9B%9E%E5%BF%86/BEG']],
     [['6-16黑兔子,白兔子行动前', '/w/6-16_%E9%BB%91%E5%85%94%E5%AD%90,%E7%99%BD%E5%85%94%E5%AD%90/BEG'], ['6-17冬逝行动后', '/w/6-17_%E5%86%AC%E9%80%9D/END']]]
for aa in c:
	a = aa[0]
	n =aa[1][1]
	n=make_filename(n)
	make_html_all(a[1],a[0],next_page=n,translation=True, update_html=True,force_translate=True)

6-12冰原之雪行动前 fetched. now translating..


In [207]:
soup = html_to_soup('/w/7-2_%E5%88%AB%E7%A6%BB%E4%B9%8B%E5%A4%9C/BEG')
df = make_df(soup)
soup.find('script', {'id': 'datas_txt', 'type': 'csv'})

./csv/4-2-END.csv
⇒ 4-2雨中漫步行动后　-　雨中散歩 戦闘後 saved to 4-2-END.html


In [66]:
from pathlib import Path
import pandas as pd

# ディレクトリ 'soup' 以下の全ファイルをループ処理
soup_dir = Path('soup')

# 全ファイルから name 列のユニークなパターンを収集するセット
d= defaultdict(int)

for file_path in soup_dir.rglob('*'):
	if str(file_path)[9]=='8' or str(file_path)[10]=='8':
		if file_path.is_file():
			with open(file_path,'r') as f:
				h = f.read()
			if not h:
				continue
			soup = BeautifulSoup(h,'html.parser')
			df = make_df(soup)
			if 'name' in df.columns:
				for name in df['name'].dropna():
					d[name] += 1


d = sorted(d.items(), key=lambda x: x[1], reverse=True)

l = {k:v for k,v in d if ('char_' not in k and 'avg_' not in k and 'eb_' not in k and '$' not in k  )}


In [67]:
l

{'塔露拉': 715,
 '凯尔希': 481,
 '阿丽娜': 271,
 '盾卫': 245,
 '阿米娅': 213,
 '迷迭香': 167,
 '科西切': 150,
 '爱国者': 123,
 '“皇帝的利刃”': 106,
 '感染者': 81,
 '霜星': 76,
 '陈': 64,
 '老妇': 56,
 'W': 55,
 '迷迭香小队成员': 49,
 '罗德岛近卫干员？': 49,
 '萨卡兹雇佣兵': 45,
 '魏彦吾': 44,
 '感染者战士': 43,
 '感染者纠察队': 42,
 '罗德岛干员': 41,
 '九': 40,
 '整合运动感染者': 29,
 '老汉': 27,
 '皇帝内卫？': 26,
 '？？？': 25,
 '霍克': 23,
 '风笛': 23,
 '陈晖洁': 21,
 '极境': 18,
 '整合运动成员': 17,
 '佩特洛娃': 17,
 '幻影弩手': 14,
 '萨沙': 14,
 '牧群化萨卡兹战士': 13,
 'Pith': 8,
 '雪怪小队成员': 7,
 '伊诺': 5,
 '斐迪亚贵族': 5,
 '怯生生的孩子': 5,
 '雪怪小队队员': 5,
 '查德': 4,
 '乌萨斯当地驻军': 4,
 '乌萨斯农民': 3,
 '学院信使': 3,
 '影卫': 2,
 '沁': 2,
 '萨卡兹牧群': 2,
 'left': 1,
 '狙击干员': 1,
 '重装干员': 1,
 '术师干员': 1,
 '女性声音': 1}

In [53]:
l = [a for a in l if '$' not in a ]
l

['',
 ' 能天使',
 '12F',
 'Ace',
 'Guard',
 'Logos',
 'Mechanist',
 'Medic',
 'Misery',
 'Mon3tr',
 'Mon3tr？',
 'Outcast',
 'PRTS',
 'Pith',
 'Raidian',
 'Rosmontis',
 'Scout',
 'W',
 'W小队成员',
 'left',
 'middle',
 'npc_10002',
 'npc_10003_1',
 'right',
 '“伊比利亚人”',
 '“会计”',
 '“副官希尔”',
 '“卡西米尔人”',
 '“司机”',
 '“吊塔”',
 '“囚犯”',
 '“戈尔丁”',
 '“最后的蒸汽骑士”',
 '“校官”',
 '“灰礼帽”',
 '“灵幛”',
 '“皇帝的利刃”',
 '“矿工”',
 '“神秘观众”',
 '“老财”',
 '“茉莉”',
 '“萨尔贡人”',
 '“蒸汽甲胄”',
 '“铣痕”',
 '“雄辩家”',
 '“领袖”',
 '三角铁',
 '不满的私兵',
 '临光',
 '丽兹？',
 '乌萨斯军警',
 '乌萨斯军警队长',
 '乌萨斯农民',
 '乌萨斯当地驻军',
 '乌萨斯感染者',
 '乌萨斯民众',
 '乌萨斯青年学生',
 '九',
 '伊内丝',
 '伊斯拉姆·维特',
 '伊诺',
 '传令兵',
 '伦蒂尼姆商人',
 '伦蒂尼姆工人',
 '伦蒂尼姆工人A',
 '伦蒂尼姆工人B',
 '伦蒂尼姆工人C',
 '伦蒂尼姆工人们',
 '伦蒂尼姆市民',
 '低沉的声音',
 '佩特洛娃',
 '侍女艾尔希',
 '儿子',
 '儿童',
 '克洛维希娅',
 '克雷格',
 '全副武装的男性',
 '冷漠的感染者',
 '冷静的暴徒',
 '凛冽的声音',
 '凯利上尉',
 '凯尔希',
 '凯瑟琳',
 '凶狠的萨卡兹',
 '前城防军成员',
 '剑卫',
 '副官希尔',
 '医疗干员',
 '华法琳',
 '卡谢娜',
 '卡铎尔',
 '厄尔苏拉',
 '厨子麦克马丁',
 '双簧管',
 '受伤的士兵',
 '受伤的青年',
 '变形者集群',
 '另一个少女的声音',
 '可露希尔',
 '号角',
 '号角&风笛'

In [ ]:
{'能天使':'エクシア',


 }
